## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-23-22-50-00 +0000,wsj,Deckers Expects More Cautious Shoppers Due to ...,https://www.wsj.com/business/earnings/deckers-...
1,2025-10-23-22-48-30 +0000,startribune,"Target eliminating 1,800 positions, laying off...",https://www.startribune.com/target-layoffs-100...
2,2025-10-23-22-47-43 +0000,nypost,Zohran Mamdani dodges questions about lacklust...,https://nypost.com/2025/10/23/us-news/zohran-m...
3,2025-10-23-22-44-02 +0000,nypost,Construction worker dies after falling 60 feet...,https://nypost.com/2025/10/23/us-news/construc...
4,2025-10-23-22-39-34 +0000,nypost,Trump says he will inform Congress of plans to...,https://nypost.com/2025/10/23/us-news/trump-sa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2453/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
36,trump,58
167,new,21
219,russian,13
392,sanctions,13
12,off,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
332,2025-10-23-03-10-21 +0000,nypost,Entire White House East Wing will be demolishe...,https://nypost.com/2025/10/22/us-news/entire-w...,139
356,2025-10-23-00-34-00 +0000,wsj,The White House is tearing down the entire Eas...,https://www.wsj.com/politics/policy/white-hous...,120
82,2025-10-23-20-01-18 +0000,nypost,Here are the corporations and donors footing t...,https://nypost.com/2025/10/23/us-news/here-are...,116
241,2025-10-23-11-45-10 +0000,bbc,Trump says Putin talks 'don't go anywhere' as ...,https://www.bbc.com/news/articles/cd6758pn6ylo...,116
71,2025-10-23-20-25-31 +0000,nyt,Trump’s Sanctions on Russian Oil Sector Ratche...,https://www.nytimes.com/2025/10/23/us/politics...,116


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
332,139,2025-10-23-03-10-21 +0000,nypost,Entire White House East Wing will be demolishe...,https://nypost.com/2025/10/22/us-news/entire-w...
71,58,2025-10-23-20-25-31 +0000,nyt,Trump’s Sanctions on Russian Oil Sector Ratche...,https://www.nytimes.com/2025/10/23/us/politics...
37,50,2025-10-23-21-36-02 +0000,nypost,X-ray poker tables allegedly used by NYC mafia...,https://nypost.com/2025/10/23/us-news/x-ray-po...
41,50,2025-10-23-21-24-00 +0000,wsj,Lawmakers failed to provide a last-minute repr...,https://www.wsj.com/politics/policy/air-traffi...
336,49,2025-10-23-02-41-32 +0000,nypost,Mom of 4 shot dead by ‘obsessed’ ex-boyfriend ...,https://nypost.com/2025/10/22/world-news/young...
244,42,2025-10-23-11-35-08 +0000,cbc,King Charles becomes the first British royal t...,https://www.cbc.ca/news/world/rome-vatican-leo...
344,41,2025-10-23-02-00-00 +0000,wsj,Mamdani and Cuomo Trade Barbs in Tense New Yor...,https://www.wsj.com/us-news/mamdani-and-cuomo-...
327,40,2025-10-23-04-01-00 +0000,wsj,U.S. Widens Campaign Against Alleged Drug Boat...,https://www.wsj.com/world/americas/u-s-widens-...
39,39,2025-10-23-21-29-34 +0000,latimes,"Trump calls off San Francisco 'surge,' but Eas...",https://www.latimes.com/california/story/2025-...
209,35,2025-10-23-14-07-46 +0000,nyt,Three Offers in One Month: Paramount’s Secret ...,https://www.nytimes.com/2025/10/22/business/de...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
